# SIT205 Thinking Systems and Cognition Science - Assignment 2

## Group: Philip Castiglione () and Warwick Smith (215239649)

## Topic 1: Text Analysis

Introductory comments regarding theme of analysis, data source used (e.g. Twitter), etc.

### Part 1 - Document Collection

In [2]:
# Standard Libaries
import os
import pickle
import json

# Additional libraries
import twitter                    # python-twitter API
from dotenv import load_dotenv    # for management of twitter credentials

In [3]:
load_dotenv()

True

In [4]:
#Utilise external environment to source Twitter API authetication credentials
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_TOKEN_KEY = os.getenv("ACCESS_TOKEN_KEY")
ACCESS_TOKEN_SECRET = os.getenv("ACCESS_TOKEN_SECRET")

#print(CONSUMER_KEY)
#print(CONSUMER_SECRET)
#print(ACCESS_TOKEN_KEY)
#print(ACCESS_TOKEN_SECRET)

# WARNING: do not commit this file to git with these values printed to a cell's output

In [5]:
# Create python-twitter API instance
api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
#print(api.VerifyCredentials())

In [6]:
# Twitter hashtags for searching 
hashtags = ["libspill", "auspol", "MalcolmTurnbull"]

In [91]:
#function for tweet collection
def collect_tweets(api, hashtag, batch_max, total_count):
    """
    Function to collects tweets using the python-twitter GetSearch API
    
    api:         Twitter API instance
    hashtag:     search hashtag
    batch_max:   maximum number of tweets to collect per each request
    total_count: maximum number of tweets to collect in total
    """
    
    # the collection of tweets to be returned
    tweets = []
    batch_max = str(batch_max)
    
    #collect the first batch of tweets
    #add tweet_mode=extended to the below
    results = api.GetSearch(term=hashtag, result_type="recent", lang="en", 
                            count=batch_max, return_json=True)
    
    #add results to list
    tweets += results['statuses']
    
    # find the the relevant starting ID for the next search
    ids = [tweet['id'] for tweet in tweets]
    max_tweet_id = str(min(ids)-1)   #reduced the minimum ID by 1 to remove duplication of tweets at the start/end of batches.

    # collect the remaining batches in the total_count
    while len(tweets) < total_count:
        
        print("{} tweets collected for hashtag {}. Most recent tweeted at {}".format(
            len(tweets), hashtag, tweets[len(tweets)-1]['created_at']))
        
        #add tweet_mode=extended to the below
        results = api.GetSearch(term=hashtag, result_type="recent", lang="en", 
                                count=batch_max, return_json=True, max_id=max_tweet_id)
        tweets += results['statuses']
        ids = [tweet['id'] for tweet in tweets]
        max_tweet_id = str(min(ids)-1)   #reduced the minimum ID by 1 to remove duplication of tweets at the start/end of batches.
        
    print("{} tweets collected for hashtag {}. Most recent tweeted at {}".format(
            len(tweets), hashtag, tweets[len(tweets)-1]['created_at']))
    return tweets

In [93]:
# Call collect_tweets function for the chosen hashtags
hash1_tweets = collect_tweets(api, hashtags[0], 100, 600)
hash2_tweets = collect_tweets(api, hashtags[1], 100, 600)
hash3_tweets = collect_tweets(api, hashtags[2], 100, 300)
tweet_collection = [hash1_tweets, hash2_tweets, hash3_tweets]

TwitterError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [68]:
print("Tweet data type: {}\t\t".format(type(hash1_tweets[0])))
print("\nTweet keys:\n")
print(hash1_tweets[0].keys())

Tweet data type: <class 'dict'>		

Tweet keys:

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])


In [69]:
#pickle all tweet sets in the tweet_collection
for i in range(len(hashtags)):
    name = hashtags[i]
    cache_filename = f"cached_tweets_{name}.pkl"
    with open(cache_filename, 'wb') as f:
        pickle.dump(tweet_collection[i], f)

In [79]:
#test load all pickled tweet sets into a new tweet_collection
loaded_tweet_collection = []
for i in range(len(hashtags)):
    name = hashtags[i]
    cache_filename = f"cached_tweets_{name}.pkl"
    with open(cache_filename, 'rb') as f:
        loaded_tweet_collection.append(pickle.load(f))

print("Number of hashtag collections:\t\t {}".format(len(loaded_tweet_collection)))
print("Tweet count per hashtag collection:\t {}, {}, {}".format(len(loaded_tweet_collection[0]),
                                                                len(loaded_tweet_collection[1]),
                                                                len(loaded_tweet_collection[2])))

Number of hashtag collections:		 3
Tweet count per hashtag collection:	 600, 600, 600


### Part 2 - Document Cleaning

In [80]:
#code here

### Part 3 - Watson NLU

In [81]:
#code here

### Part 4 - Analysis of Watson Output

In [ ]:
#code here